In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# dimensions of our images.
img_width, img_height = 299, 299

train_data_dir = './pokemon_dataset' #contains two classes cats and dogs
validation_data_dir = './pokemon_dataset_test' #contains two classes cats and dogs

nb_train_samples = 13000
nb_validation_samples = 800
nb_epoch = 10

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(150, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(rescale=1./255)#,
 #       shear_range=0.2,
 #       zoom_range=0.2,
 #       horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

print "start history model"
history = model.fit_generator(
    train_generator,
    epochs=nb_epoch,
    steps_per_epoch=1000,
    validation_data=validation_generator,
    validation_steps=800) #1020

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
#model.fit_generator(...)
# fine-tune the model
model.fit_generator(
        train_generator,
        steps_per_epoch=1000,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples)

model.save("inception_retrained_model1")

Using TensorFlow backend.
/home/wolf/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


Found 13279 images belonging to 150 classes.
Found 2388 images belonging to 150 classes.
start history model
Epoch 1/10


/home/wolf/anaconda2/lib/python2.7/site-packages/PIL/Image.py:885: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


 382/1000 [==========>...................] - ETA: 383s - loss: 4.5621 - acc: 0.0825

/home/wolf/anaconda2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:709: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


1000/1000 [==============================] - 815s - loss: 3.5916 - acc: 0.2294 - val_loss: 2.2009 - val_acc: 0.4901
Epoch 2/10
1000/1000 [==============================] - 705s - loss: 2.1444 - acc: 0.4843 - val_loss: 1.5303 - val_acc: 0.6238
Epoch 3/10
1000/1000 [==============================] - 714s - loss: 1.6620 - acc: 0.5804 - val_loss: 1.1133 - val_acc: 0.7108
Epoch 4/10
1000/1000 [==============================] - 707s - loss: 1.3676 - acc: 0.6386 - val_loss: 0.8437 - val_acc: 0.7762
Epoch 5/10
1000/1000 [==============================] - 722s - loss: 1.1619 - acc: 0.6849 - val_loss: 0.6069 - val_acc: 0.8339
Epoch 6/10
1000/1000 [==============================] - 716s - loss: 0.9858 - acc: 0.7218 - val_loss: 0.4951 - val_acc: 0.8596
Epoch 7/10
1000/1000 [==============================] - 702s - loss: 0.8660 - acc: 0.7494 - val_loss: 0.3978 - val_acc: 0.8779
Epoch 8/10
1000/1000 [==============================] - 699s - loss: 0.7731 - acc: 0.7733 - val_loss: 0.3739 - val_acc: 0.

Epoch 1/10
1000/1000 [==============================] - 905s - loss: 0.2714 - val_loss: 0.0650
Epoch 2/10
1000/1000 [==============================] - 902s - loss: 0.1445 - val_loss: 0.0548
Epoch 3/10
1000/1000 [==============================] - 902s - loss: 0.1102 - val_loss: 0.0471
Epoch 4/10
1000/1000 [==============================] - 924s - loss: 0.0948 - val_loss: 0.0506
Epoch 5/10
1000/1000 [==============================] - 940s - loss: 0.0832 - val_loss: 0.0476
Epoch 6/10
1000/1000 [==============================] - 942s - loss: 0.0748 - val_loss: 0.0414
Epoch 7/10
1000/1000 [==============================] - 1013s - loss: 0.0727 - val_loss: 0.0392
Epoch 8/10
1000/1000 [==============================] - 994s - loss: 0.0682 - val_loss: 0.0401
Epoch 9/10
1000/1000 [==============================] - 959s - loss: 0.0649 - val_loss: 0.0390
Epoch 10/10
1000/1000 [==============================] - 966s - loss: 0.0598 - val_loss: 0.0420


In [130]:
from keras.models import load_model
nb_epoch = 3
model = load_model("inception_retrained_model3")

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=40,
    fill_mode='nearest',
    shear_range=0.2,)
#,
 #       shear_range=0.2,
 #       zoom_range=0.2,
 #       horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='categorical'
)

print "start history model"
history = model.fit_generator(
    train_generator,
    epochs=nb_epoch,
    steps_per_epoch=1000,
    validation_data=validation_generator,
    validation_steps=800) #1020

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
#model.fit_generator(...)
# fine-tune the model
model.fit_generator(
        train_generator,
        steps_per_epoch=1000,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples)

model.save("inception_retrained_model3")

Found 13279 images belonging to 150 classes.
Found 2388 images belonging to 150 classes.
start history model
Epoch 1/3


ResourceExhaustedError: OOM when allocating tensor with shape[16,64,147,147]
	 [[Node: conv2d_3_9/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](activation_2_9/Relu, conv2d_3_9/kernel/read)]]
	 [[Node: loss_17/mul/_26983 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_8453_loss_17/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'conv2d_3_9/convolution', defined at:
  File "/home/wolf/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/wolf/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/wolf/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <modul

In [73]:
classes = train_generator.class_indices


In [75]:
res = dict((v,k) for k,v in classes.iteritems())

In [101]:
%ls

18.jpg                          Pikachu.jpg
58.jpg                          pika.jpg
abra.jpeg                       pokemon2.csv
author-quote.txt                pokemon.csv
billboard_lyrics_1964-2015.txt  pokemon_dataset/
Cartpole DQN-Copy1.ipynb        pokemon_dataset_test/
Cartpole DQN.ipynb              python_gen/
division.ipynb                  pythontxt.txt
downloader.ipynb                README.md
house prices.ipynb              rltrader.h5
imagenet.ipynb                  RL TRADER.ipynb
inception_retrained_model1      TRADER.ipynb
inception_retrained_model2      Untitled1.ipynb
kaggle_house/                   Untitled2.ipynb
model.h5                        Untitled3.ipynb
model_pend.h5                   Untitled4-Copy1.ipynb
output.txt                      Untitled5.ipynb
pendulum DQN.ipynb              Untitled.ipynb


In [142]:
from keras.preprocessing import image
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
from PIL import Image
import urllib, cStringIO

file = cStringIO.StringIO(urllib.urlopen("https://i.pinimg.com/736x/2f/a6/fa/2fa6fa2b399f3d85d8f403d4ac5ec666--centro-pokemon-amigurumi-patterns.jpg").read())
img_path = file
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
# y_classes = preds.argmax(axis=-1)
predictions = preds[0].argsort(axis=-1)[-3:]
print predictions
for x in predictions:
    print preds[0][x]
    print res[x]

[102 110  98]
0.0478194
pikachu
0.195985
psyduck
0.689114
persian


In [143]:
model.save("inception_retrained_model4")